# **Laboratorio estadístico [Análisis de Datos de Ventas en una Tienda Online para Optimización de Inventarios]**
## **Daniel Alejandro García Gonzalez**

<div align="left">
    <img src="Logo.svg" width="30%">


***


## **Introducción**
> Este análisis busca optimizar la gestión de inventarios en una tienda de repostería, utilizando datos históricos de ventas. La tienda ofrece una variedad de productos, como postres y futuramente pasteles, galletas y cupcakes, cada uno con diferentes niveles de demanda. A través de este análisis, se pretende identificar patrones en las ventas y tomar decisiones informadas para mejorar la disponibilidad de productos y reducir el desperdicio.




## **Planteamiento del Problema**
> La tienda de repostería "L'atelier" ha enfrentado problemas con la falta de inventario de algunos productos populares y el exceso de otros que no se venden tan bien. Esto ha generado pérdidas tanto por la falta de productos para satisfacer la demanda como por el desperdicio de productos perecederos. El objetivo de este análisis es entender mejor la demanda de los diferentes productos y optimizar los niveles de inventario.

## **Objetivo General**
> Optimizar los niveles de inventario de la tienda de repostería para mejorar la satisfacción del cliente y reducir las pérdidas por productos no vendidos.


## **Objetivos Específicos**
1. **Variables Cuantitativas:**
   - **Continuas:**
     - **Variable 1:** *[Precio del producto: Variar los precios puede afectar la demanda.]*
     - **Variable 2:** *[Unidades vendidas: Número total de unidades vendidas por producto.]*
   - **Discreta:**
     - **Variable 3:** *[Días con stock: Número de días en los que el producto estuvo disponible en el inventario.]*

2. **Variables Cualitativas:**
   - **Nominales:**
     - **Variable 1:** *[Categoría del producto (nominal): Pasteles, galletas, cupcakes, etc.]*
     - **Variable 2:** *[Tipo de cliente (nominal): Nuevos, recurrentes.]*
   - **Ordinal:**
     - **Variable 3:** *Satisfacción del cliente (ordinal): Bajo, medio, alto.*

> Explicar cómo cada una de estas variables está relacionada con el problema que se plantea.


## **Metodología**
Fuente de datos: Se obtendrán exactamente 100 registros de ventas de productos en la tienda durante un periodo de 30 días. Estos datos incluirán información sobre el tipo de producto, cantidad vendida, precio y satisfacción del cliente.
### **Fuente de Datos**
> Justificación: Los datos provienen del sistema de ventas de la tienda y reflejan las ventas reales.
***



In [30]:
# Importación de bibliotecas necesarias
import numpy as np  # Biblioteca para operaciones numéricas y manejo de arrays
import pandas as pd  # Biblioteca para manipulación y análisis de datos
from scipy import stats  # Módulo de SciPy para funciones estadísticas
from scipy.special import factorial
from scipy.stats import chi2_contingency, poisson, norm, expon
import plotly.io as pio  # Módulo de Plotly para gestionar la visualización
import plotly.express as px  # Módulo de Plotly para crear gráficos de manera sencilla
import plotly.graph_objects as go


# Configuración del renderizador de Plotly, Esto permite que los gráficos se muestren en Jupyter
pio.renderers.default = 'notebook_connected'

In [31]:
# Generar datos aleatorios para probar que todo funcione
np.random.seed(42)  # Para reproducibilidad

# Definir precios fijos para cada producto
precios = {
    'Cheesecake': 9000,
    'Carlota': 8000,
    'Panacota': 10000
}

# Crear un diccionario con datos aleatorios
productos = np.random.choice(list(precios.keys()), size=100)  # Seleccionar aleatoriamente un producto de la lista

data = {
    'Producto': productos,  # Utilizar los productos seleccionados
    'Tipo_Cliente': np.random.choice(['Nuevo', 'Recurrente'], size=100),  # Seleccionar aleatoriamente el tipo de cliente
    'Satisfaccion': np.random.choice(['Bajo', 'Medio', 'Alto'], size=100, p=[0.1, 0.4, 0.5]),  # Seleccionar satisfacción con probabilidades específicas
    'Precio': [precios[producto] for producto in productos],  # Asignar el precio correspondiente según el producto
    'Unidades_Vendidas': np.random.poisson(lam=5, size=100),  # Generar unidades vendidas siguiendo una distribución de Poisson
    'Dias_Stock': np.random.randint(1, 31, size=100)  # Generar días de stock aleatorios entre 1 y 30
}

# Crear un DataFrame de pandas a partir del diccionario de datos
df = pd.DataFrame(data)

# Mostrar las primeras 5 filas del DataFrame para verificar que los datos se han generado correctamente
df.head()

,Producto,Tipo_Cliente,Satisfaccion,Precio,Unidades_Vendidas,Dias_Stock
0,Panacota,Recurrente,Alto,10000,3,9
1,Cheesecake,Recurrente,Medio,9000,8,3
2,Panacota,Recurrente,Medio,10000,3,20
3,Panacota,Nuevo,Medio,10000,6,25
4,Cheesecake,Nuevo,Medio,9000,9,26


## **Análisis Descriptivo de Datos**

### **Análisis Univariado**

#### **Tabla de Entrada de Datos**

- **Producto**: Categoría del producto (Pastel, Galleta, Cupcake).
- **Tipo_Cliente**: Tipo de cliente (Nuevo, Recurrente).
- **Satisfaccion**: Nivel de satisfacción del cliente (Bajo, Medio, Alto).
- **Precio**: Precio del producto.
- **Unidades_Vendidas**: Cantidad de unidades vendidas.
- **Dias_Stock**: Número de días que el producto estuvo en stock.

#### **Tablas de Frecuencia para Variables Cualitativas**


In [32]:
# Calcular la frecuencia de cada categoría en la columna 'Producto'
frecuencia_producto = df['Producto'].value_counts()

# Calcular la frecuencia de cada categoría en la columna 'Tipo_Cliente'
frecuencia_cliente = df['Tipo_Cliente'].value_counts()

# Calcular la frecuencia de cada categoría en la columna 'Satisfaccion'
frecuencia_satisfaccion = df['Satisfaccion'].value_counts()

# Imprimir las frecuencias para cada variable cualitativa
print("Frecuencia por Producto:")
print(frecuencia_producto)  # Mostrar la frecuencia de productos

print("\nFrecuencia por Tipo de Cliente:")
print(frecuencia_cliente)  # Mostrar la frecuencia de tipos de cliente

print("\nFrecuencia por Satisfacción:")
print(frecuencia_satisfaccion)  # Mostrar la frecuencia de satisfacción


Frecuencia por Producto:
Producto
Carlota       36
Cheesecake    33
Panacota      31
Name: count, dtype: int64

Frecuencia por Tipo de Cliente:
Tipo_Cliente
Nuevo         56
Recurrente    44
Name: count, dtype: int64

Frecuencia por Satisfacción:
Satisfaccion
Alto     51
Medio    39
Bajo     10
Name: count, dtype: int64


#### **Gráficos Circulares y de Barras para Variables Cualitativas**


In [33]:

# Gráfico circular para "Producto"
fig_pie_producto = px.pie(
    frecuencia_producto,  # Datos de frecuencia para el gráfico
    names=frecuencia_producto.index,  # Nombres de las categorías (productos)
    values=frecuencia_producto.values,  # Valores de frecuencia correspondientes
    title='Distribución de Productos',  # Título del gráfico
    color_discrete_sequence=px.colors.qualitative.Set3  # Paleta de colores para el gráfico
)

# Mostrar el gráfico circular de productos
fig_pie_producto.show()

# Gráfico circular para "Tipo de Cliente"
fig_pie_cliente = px.pie(
    frecuencia_cliente,  # Datos de frecuencia para el gráfico
    names=frecuencia_cliente.index,  # Nombres de las categorías (tipos de cliente)
    values=frecuencia_cliente.values,  # Valores de frecuencia correspondientes
    title='Distribución por Tipo de Cliente',  # Título del gráfico
    color_discrete_sequence=px.colors.qualitative.Set3  # Paleta de colores para el gráfico
)

# Mostrar el gráfico circular de tipo de cliente
fig_pie_cliente.show()

# Gráfico circular para "Satisfacción"
fig_pie_satisfaccion = px.pie(
    frecuencia_satisfaccion,  # Datos de frecuencia para el gráfico
    names=frecuencia_satisfaccion.index,  # Nombres de las categorías (niveles de satisfacción)
    values=frecuencia_satisfaccion.values,  # Valores de frecuencia correspondientes
    title='Distribución por Nivel de Satisfacción',  # Título del gráfico
    color_discrete_sequence=px.colors.qualitative.Set3  # Paleta de colores para el gráfico
)

# Mostrar el gráfico circular de satisfacción
fig_pie_satisfaccion.show()

In [34]:
# Gráfico de barras para "Producto"
fig_producto = px.bar(
    frecuencia_producto,  # Datos de frecuencia para el gráfico
    x=frecuencia_producto.index,  # Nombres de las categorías (productos) en el eje x
    y=frecuencia_producto.values,  # Valores de frecuencia correspondientes en el eje y
    title='Frecuencia de Productos',  # Título del gráfico
    labels={'x': 'Producto', 'y': 'Frecuencia'},  # Etiquetas para los ejes
    color=frecuencia_producto.index,  # Color basado en las categorías (productos)
    color_discrete_sequence=px.colors.qualitative.Set3  # Paleta de colores para el gráfico
)

# Mostrar el gráfico de barras de productos
fig_producto.show()

# Gráfico de barras para "Tipo de Cliente"
fig_cliente = px.bar(
    frecuencia_cliente,  # Datos de frecuencia para el gráfico
    x=frecuencia_cliente.index,  # Nombres de las categorías (tipos de cliente) en el eje x
    y=frecuencia_cliente.values,  # Valores de frecuencia correspondientes en el eje y
    title='Frecuencia por Tipo de Cliente',  # Título del gráfico
    labels={'x': 'Tipo de Cliente', 'y': 'Frecuencia'},  # Etiquetas para los ejes
    color=frecuencia_cliente.index,  # Color basado en las categorías (tipos de cliente)
    color_discrete_sequence=px.colors.qualitative.Set3  # Paleta de colores para el gráfico
)

# Mostrar el gráfico de barras de tipo de cliente
fig_cliente.show()

# Gráfico de barras para "Satisfacción"
fig_satisfaccion = px.bar(
    frecuencia_satisfaccion,  # Datos de frecuencia para el gráfico
    x=frecuencia_satisfaccion.index,  # Nombres de las categorías (niveles de satisfacción) en el eje x
    y=frecuencia_satisfaccion.values,  # Valores de frecuencia correspondientes en el eje y
    title='Frecuencia por Satisfacción',  # Título del gráfico
    labels={'x': 'Nivel de Satisfacción', 'y': 'Frecuencia'},  # Etiquetas para los ejes
    color=frecuencia_satisfaccion.index,  # Color basado en las categorías (niveles de satisfacción)
    color_discrete_sequence=px.colors.qualitative.Set3  # Paleta de colores para el gráfico
)

# Mostrar el gráfico de barras de satisfacción
fig_satisfaccion.show()


#### **Tablas de Frecuencia para Variables Cuantitativas**


##### **Tabla de frecuencia no agrupada**


In [35]:
# Tabla de frecuencia no agrupada para 'Precio'
frecuencia_precio = df['Precio'].value_counts().sort_index()
print("Tabla de frecuencia no agrupada para 'Precio':\n", frecuencia_precio)

# Tabla de frecuencia no agrupada para 'Unidades_Vendidas'
frecuencia_unidades = df['Unidades_Vendidas'].value_counts().sort_index()
print("\nTabla de frecuencia no agrupada para 'Unidades_Vendidas':\n", frecuencia_unidades)

# Tabla de frecuencia no agrupada para 'Dias_Stock'
frecuencia_dias_stock = df['Dias_Stock'].value_counts().sort_index()
print("\nTabla de frecuencia no agrupada para 'Dias_Stock':\n", frecuencia_dias_stock)

Tabla de frecuencia no agrupada para 'Precio':
 Precio
8000     36
9000     33
10000    31
Name: count, dtype: int64

Tabla de frecuencia no agrupada para 'Unidades_Vendidas':
 Unidades_Vendidas
1      2
2     11
3     18
4     14
5     13
6     17
7     11
8      6
9      6
10     1
11     1
Name: count, dtype: int64

Tabla de frecuencia no agrupada para 'Dias_Stock':
 Dias_Stock
1     5
2     4
3     2
4     1
6     3
7     5
8     3
9     2
10    2
11    3
12    1
13    2
14    2
15    1
16    1
17    8
18    6
19    7
20    4
21    2
22    5
23    4
24    3
25    2
26    2
27    5
28    5
29    5
30    5
Name: count, dtype: int64


##### **Tabla de frecuencia agrupada**

In [36]:
# Definir los intervalos para agrupar las variables en 5 bins
intervalos_precio = pd.cut(df['Precio'], bins=5)  # Agrupar precios en 5 intervalos
intervalos_unidades = pd.cut(df['Unidades_Vendidas'], bins=5)  # Agrupar unidades vendidas en 5 intervalos
intervalos_dias_stock = pd.cut(df['Dias_Stock'], bins=5)  # Agrupar días de stock en 5 intervalos

# Tabla de frecuencia agrupada para 'Precio'
frecuencia_agrupada_precio = intervalos_precio.value_counts().sort_index()  # Contar frecuencias en los intervalos
print("\nTabla de frecuencia agrupada para 'Precio':\n", frecuencia_agrupada_precio)

# Tabla de frecuencia agrupada para 'Unidades_Vendidas'
frecuencia_agrupada_unidades = intervalos_unidades.value_counts().sort_index()  # Contar frecuencias en los intervalos
print("\nTabla de frecuencia agrupada para 'Unidades_Vendidas':\n", frecuencia_agrupada_unidades)

# Tabla de frecuencia agrupada para 'Dias_Stock'
frecuencia_agrupada_dias_stock = intervalos_dias_stock.value_counts().sort_index()  # Contar frecuencias en los intervalos
print("\nTabla de frecuencia agrupada para 'Dias_Stock':\n", frecuencia_agrupada_dias_stock)



Tabla de frecuencia agrupada para 'Precio':
 Precio
(7998.0, 8400.0]     36
(8400.0, 8800.0]      0
(8800.0, 9200.0]     33
(9200.0, 9600.0]      0
(9600.0, 10000.0]    31
Name: count, dtype: int64

Tabla de frecuencia agrupada para 'Unidades_Vendidas':
 Unidades_Vendidas
(0.99, 3.0]    31
(3.0, 5.0]     27
(5.0, 7.0]     28
(7.0, 9.0]     12
(9.0, 11.0]     2
Name: count, dtype: int64

Tabla de frecuencia agrupada para 'Dias_Stock':
 Dias_Stock
(0.971, 6.8]    15
(6.8, 12.6]     16
(12.6, 18.4]    20
(18.4, 24.2]    25
(24.2, 30.0]    24
Name: count, dtype: int64


In [37]:
# Medidas de tendencia central y dispersión

# Calcular medidas para 'Precio'
medidas_precio = df['Precio'].describe()  # Obtiene medidas estadísticas descriptivas
print("Medidas para Precio:")
print(medidas_precio)  # Imprime las medidas para 'Precio'

# Calcular medidas para 'Unidades_Vendidas'
medidas_unidades = df['Unidades_Vendidas'].describe()  # Obtiene medidas estadísticas descriptivas
print("\nMedidas para Unidades Vendidas:")
print(medidas_unidades)  # Imprime las medidas para 'Unidades Vendidas'

Medidas para Precio:
count      100.000000
mean      8950.000000
std        821.122661
min       8000.000000
25%       8000.000000
50%       9000.000000
75%      10000.000000
max      10000.000000
Name: Precio, dtype: float64

Medidas para Unidades Vendidas:
count    100.000000
mean       5.010000
std        2.222452
min        1.000000
25%        3.000000
50%        5.000000
75%        6.250000
max       11.000000
Name: Unidades_Vendidas, dtype: float64


#### **Histogramas para Variables Cuantitativas**

##### **Histograma agrupado**

In [38]:
# Histograma agrupado para Precio de productos
fig_hist_precio = px.histogram(df,
                              x='Precio',
                              nbins=10,  # Número de bins (Intervalos)
                              title='Distribución del Precio de los Productos',
                              labels={'Precio': 'Precio'},  # Etiquetas para el eje x
                              color_discrete_sequence=px.colors.qualitative.Set3)  # Paleta de colores
fig_hist_precio.show()  # Mostrar el histograma

# Histograma agrupado para Unidades Vendidas
fig_hist_unidades_vendidas = px.histogram(df,
                                         x='Unidades_Vendidas',
                                         nbins=10,  # Número de bins (Intervalos)
                                         title='Distribución de Unidades Vendidas',
                                         labels={'Unidades_Vendidas': 'Unidades Vendidas'},  # Etiquetas para el eje x
                                         color_discrete_sequence=px.colors.qualitative.Set3)  # Paleta de colores
fig_hist_unidades_vendidas.show()  # Mostrar el histograma

# Histograma agrupado para Días con Stock
fig_hist_dias_stock = px.histogram(df,
                                  x='Dias_Stock',
                                  nbins=10,  # Número de bins (Intervalos)
                                  title='Distribución de Días con Stock',
                                  labels={'Dias_Stock': 'Dias con Stock'},  # Etiquetas para el eje x
                                  color_discrete_sequence=px.colors.qualitative.Set3)  # Paleta de colores
fig_hist_dias_stock.show()  # Mostrar el histograma

##### **Histograma no agrupado**

In [39]:
# Histograma no agrupado para Precio de productos
fig_hist_precio_no_agrupado = px.histogram(df,
                                         x='Precio',
                                         nbins=len(df['Precio'].unique()),  # Intervalos pequeños (uno por valor único)
                                         title='Distribución No Agrupada del Precio de los Productos',
                                         labels={'Precio': 'Precio'},
                                         color_discrete_sequence=px.colors.qualitative.Set3)
fig_hist_precio_no_agrupado.show() 

# Histograma no agrupado para Unidades_Vendidas
fig_hist_unidades_vendidas_no_agrupado = px.histogram(df,
                                                   x='Unidades_Vendidas',
                                                   nbins=len(df['Unidades_Vendidas'].unique()),  # Intervalos pequeños
                                                   title='Distribución No Agrupada de Unidades Vendidas',
                                                   labels={'Unidades_Vendidas': 'Unidades Vendidas'},
                                                   color_discrete_sequence=px.colors.qualitative.Set3)
fig_hist_unidades_vendidas_no_agrupado.show()

# Histograma no agrupado para Días_Stock
fig_hist_dias_stock_no_agrupado = px.histogram(df,
                                             x='Dias_Stock',
                                             nbins=len(df['Dias_Stock'].unique()),  # Intervalos pequeños
                                             title='Distribución No Agrupada de Días con Stock',
                                             labels={'Dias_Stock': 'Días con Stock'},
                                             color_discrete_sequence=px.colors.qualitative.Set3)
fig_hist_dias_stock_no_agrupado.show()

#### **Medidas de Tendencia Central para Variables Cuantitativas**

In [40]:
# Cálculo de Medidas de Tendencia Central
medidas_tendencia_central = pd.DataFrame({
    'Variable': ['Precio', 'Unidades Vendidas', 'Días de Stock'],  # Nombres de las variables
    'Media': [df['Precio'].mean(), df['Unidades_Vendidas'].mean(), df['Dias_Stock'].mean()],  # Cálculo de la media
    'Mediana': [df['Precio'].median(), df['Unidades_Vendidas'].median(), df['Dias_Stock'].median()],  # Cálculo de la mediana
    'Moda': [df['Precio'].mode()[0], df['Unidades_Vendidas'].mode()[0], df['Dias_Stock'].mode()[0]]  # Cálculo de la moda
})

# Mostrar las medidas de tendencia central
print(medidas_tendencia_central)


            Variable    Media  Mediana  Moda
0             Precio  8950.00   9000.0  8000
1  Unidades Vendidas     5.01      5.0     3
2      Días de Stock    17.13     18.0    17


#### **Análisis de Simetría y Regla Empírica**

In [41]:
# Cálculo de sesgo y kurtosis para cada variable
sesgo_precio = df['Precio'].skew()  # Calcular el sesgo de la variable 'Precio'
kurtosis_precio = df['Precio'].kurtosis()  # Calcular la kurtosis de la variable 'Precio'

sesgo_unidades_vendidas = df['Unidades_Vendidas'].skew()  # Calcular el sesgo de 'Unidades Vendidas'
kurtosis_unidades_vendidas = df['Unidades_Vendidas'].kurtosis()  # Calcular la kurtosis de 'Unidades Vendidas'

sesgo_dias_stock = df['Dias_Stock'].skew()  # Calcular el sesgo de 'Días de Stock'
kurtosis_dias_stock = df['Dias_Stock'].kurtosis()  # Calcular la kurtosis de 'Días de Stock'

# Crear un DataFrame para almacenar los resultados del análisis de simetría
analisis_simetria = pd.DataFrame({
    'Variable': ['Precio', 'Unidades Vendidas', 'Días de Stock'],  # Nombres de las variables
    'Sesgo': [sesgo_precio, sesgo_unidades_vendidas, sesgo_dias_stock],  # Valores de sesgo
    'Kurtosis': [kurtosis_precio, kurtosis_unidades_vendidas, kurtosis_dias_stock]  # Valores de kurtosis
})

# Mostrar el análisis de simetría
print(analisis_simetria)

# Verificación de la Regla Empírica
# Calcular la media y la desviación estándar para cada variable
media_precio = df['Precio'].mean()  # Media de 'Precio'
std_precio = df['Precio'].std()  # Desviación estándar de 'Precio'

media_unidades_vendidas = df['Unidades_Vendidas'].mean()  # Media de 'Unidades Vendidas'
std_unidades_vendidas = df['Unidades_Vendidas'].std()  # Desviación estándar de 'Unidades Vendidas'

media_dias_stock = df['Dias_Stock'].mean()  # Media de 'Días de Stock'
std_dias_stock = df['Dias_Stock'].std()  # Desviación estándar de 'Días de Stock'

# Cálculo de los límites para la Regla Empírica (1 y 2 desviaciones estándar)
limite_1_precio = (media_precio - std_precio, media_precio + std_precio)  # Límites para 1 desviación estándar de 'Precio'
limite_2_precio = (media_precio - 2 * std_precio, media_precio + 2 * std_precio)  # Límites para 2 desviaciones estándar de 'Precio'

limite_1_unidades_vendidas = (media_unidades_vendidas - std_unidades_vendidas, media_unidades_vendidas + std_unidades_vendidas)  # Límites para 1 desviación estándar de 'Unidades Vendidas'
limite_2_unidades_vendidas = (media_unidades_vendidas - 2 * std_unidades_vendidas, media_unidades_vendidas + 2 * std_unidades_vendidas)  # Límites para 2 desviaciones estándar de 'Unidades Vendidas'

limite_1_dias_stock = (media_dias_stock - std_dias_stock, media_dias_stock + std_dias_stock)  # Límites para 1 desviación estándar de 'Días de Stock'
limite_2_dias_stock = (media_dias_stock - 2 * std_dias_stock, media_dias_stock + 2 * std_dias_stock)  # Límites para 2 desviaciones estándar de 'Días de Stock'

# Resultados de la Regla Empírica
regla_empirica = {
    'Variable': ['Precio', 'Unidades Vendidas', 'Días de Stock'],  # Nombres de las variables
    '1 Desviación Estándar': [limite_1_precio, limite_1_unidades_vendidas, limite_1_dias_stock],  # Límites de 1 desviación estándar
    '2 Desviaciones Estándar': [limite_2_precio, limite_2_unidades_vendidas, limite_2_dias_stock]  # Límites de 2 desviaciones estándar
}

# Crear un DataFrame para almacenar los resultados de la regla empírica
regla_empirica_df = pd.DataFrame(regla_empirica)
print(regla_empirica_df)

            Variable     Sesgo  Kurtosis
0             Precio  0.093551 -1.511300
1  Unidades Vendidas  0.370256 -0.530682
2      Días de Stock -0.327578 -1.010175
            Variable                    1 Desviación Estándar  \
0             Precio  (8128.8773391006525, 9771.122660899347)   
1  Unidades Vendidas  (2.7875479916783514, 7.232452008321648)   
2      Días de Stock    (8.28948833694302, 25.97051166305698)   

                     2 Desviaciones Estándar  
0    (7307.754678201306, 10592.245321798695)  
1    (0.5650959833567031, 9.454904016643297)  
2  (-0.5510233261139597, 34.811023326113954)  


#### **Medidas de Variación para Variables Cuantitativas**

In [42]:
# Cálculo de medidas de variación para cada variable

# Rango, varianza y desviación estándar para la variable 'Precio'
rango_precio = df['Precio'].max() - df['Precio'].min()  # Calcular el rango (diferencia entre el valor máximo y mínimo)
varianza_precio = df['Precio'].var()  # Calcular la varianza de 'Precio'
desviacion_estandar_precio = df['Precio'].std()  # Calcular la desviación estándar de 'Precio'

# Rango, varianza y desviación estándar para la variable 'Unidades Vendidas'
rango_unidades_vendidas = df['Unidades_Vendidas'].max() - df['Unidades_Vendidas'].min()  # Calcular el rango de 'Unidades Vendidas'
varianza_unidades_vendidas = df['Unidades_Vendidas'].var()  # Calcular la varianza de 'Unidades Vendidas'
desviacion_estandar_unidades_vendidas = df['Unidades_Vendidas'].std()  # Calcular la desviación estándar de 'Unidades Vendidas'

# Rango, varianza y desviación estándar para la variable 'Días de Stock'
rango_dias_stock = df['Dias_Stock'].max() - df['Dias_Stock'].min()  # Calcular el rango de 'Días de Stock'
varianza_dias_stock = df['Dias_Stock'].var()  # Calcular la varianza de 'Días de Stock'
desviacion_estandar_dias_stock = df['Dias_Stock'].std()  # Calcular la desviación estándar de 'Días de Stock'

# Crear un DataFrame para almacenar los resultados de las medidas de variación
medidas_variacion = pd.DataFrame({
    'Variable': ['Precio', 'Unidades Vendidas', 'Días de Stock'],  # Nombres de las variables
    'Rango': [rango_precio, rango_unidades_vendidas, rango_dias_stock],  # Valores de rango
    'Varianza': [varianza_precio, varianza_unidades_vendidas, varianza_dias_stock],  # Valores de varianza
    'Desviación Estándar': [desviacion_estandar_precio, desviacion_estandar_unidades_vendidas, desviacion_estandar_dias_stock]  # Valores de desviación estándar
})

# Mostrar los resultados de las medidas de variación
print(medidas_variacion)

            Variable  Rango       Varianza  Desviación Estándar
0             Precio   2000  674242.424242           821.122661
1  Unidades Vendidas     10       4.939293             2.222452
2      Días de Stock     29      78.154646             8.840512


### **Análisis Bivariado**

#### **Tablas de Contingencia**

In [43]:
# Tabla de contingencia para 'Producto' y 'Tipo_Cliente'
tabla_contingencia_1 = pd.crosstab(df['Producto'], df['Tipo_Cliente'])
print("Tabla de contingencia entre 'Producto' y 'Tipo_Cliente':\n", tabla_contingencia_1)

# Tabla de contingencia para 'Producto' y 'Satisfaccion_Cliente'
tabla_contingencia_2 = pd.crosstab(df['Producto'], df['Satisfaccion'])
print("\nTabla de contingencia entre 'Producto' y 'Satisfaccion_Cliente':\n", tabla_contingencia_2)

Tabla de contingencia entre 'Producto' y 'Tipo_Cliente':
 Tipo_Cliente  Nuevo  Recurrente
Producto                       
Carlota          21          15
Cheesecake       15          18
Panacota         20          11

Tabla de contingencia entre 'Producto' y 'Satisfaccion_Cliente':
 Satisfaccion  Alto  Bajo  Medio
Producto                       
Carlota         18     2     16
Cheesecake      18     6      9
Panacota        15     2     14


#### **Grafico de Barras Compuestas para Tablas de Contingencia**

In [44]:
# Tabla de contingencia para 'Producto' y 'Tipo_Cliente'
tabla_contingencia_1 = pd.crosstab(df['Producto'], df['Tipo_Cliente'])

# Convertimos la tabla de contingencia a formato largo con melt()
tabla_melt_1 = tabla_contingencia_1.reset_index().melt(
    id_vars=['Producto'],
    value_vars=tabla_contingencia_1.columns,
    var_name='Tipo_Cliente',
    value_name='Cantidad'
)

# Gráfico de barras compuestas para 'Producto' y 'Tipo_Cliente'
fig_1 = px.bar(tabla_melt_1,
              x='Producto',
              y='Cantidad',
              color='Tipo_Cliente',
              title='Distribución de Productos por tipo de Cliente',
              labels={'Producto': 'Producto', 'Cantidad': 'Número de Clientes'},
              barmode='stack',  # Barra compuesta
              color_discrete_sequence=px.colors.qualitative.Set3)  # Paleta de colores personalizada
fig_1.show()

In [45]:
# Tabla de contingencia para 'Producto' y 'Satisfacción'
tabla_contingencia_2 = pd.crosstab(df['Producto'], df['Satisfaccion'])

# Convertimos la tabla de contingencia a formato largo con melt()
tabla_melt_2 = tabla_contingencia_2.reset_index().melt(
    id_vars=['Producto'],
    value_vars=tabla_contingencia_2.columns,
    var_name='Satisfaccion',
    value_name='Cantidad'
)

# Gráfico de barras compuestas para 'Producto' y 'Satisfacción'
fig_2 = px.bar(tabla_melt_2,
              x='Cantidad',
              y='Producto',
              color='Satisfaccion',
              title='Distribución de Productos por Satisfacción',
              labels={'Producto': 'Producto', 'Cantidad': 'Número de Clientes'},
              barmode='stack',  # Barra compuesta
              color_discrete_sequence=px.colors.qualitative.Set3)  # Paleta de colores personalizada
fig_2.show()

### **Aplicación de Probabilidades**

#### **Dependencia entre Producto y Tipo de cliente**

In [46]:
# Tabla de contingencia entre 'Producto' y 'Tipo de cliente'
tabla_contingencia = pd.crosstab(df['Producto'], df['Tipo_Cliente'])

# Realizar la prueba de chi-cuadrado
chi2, p, dof, expected = chi2_contingency(tabla_contingencia)

# Resultados
print(f"Chi-cuadrado: {chi2:.4f}")
print(f"Valor p: {p:.4f}")
print(f"Degrees of freedom: {dof}")
print("Valores Esperados:\n", expected)

if p < 0.05:
    print("Se rechaza la hipótesis nula: Hay dependencia entre las variables.")
else:
    print("No se rechaza la hipótesis nula: No hay evidencia de dependencia entre las variables.")

Chi-cuadrado: 2.4814
Valor p: 0.2892
Degrees of freedom: 2
Valores Esperados:
 [[20.16 15.84]
 [18.48 14.52]
 [17.36 13.64]]
No se rechaza la hipótesis nula: No hay evidencia de dependencia entre las variables.


In [47]:
# Crear la tabla de contingencia normalizada por índice
tabla_contingencia_tipo_cliente = pd.crosstab(df['Producto'], df['Tipo_Cliente'], normalize='index')

# Crear el gráfico de barras
fig_1 = px.bar(
    tabla_contingencia_tipo_cliente,
    x=tabla_contingencia_tipo_cliente.columns,
    y=tabla_contingencia_tipo_cliente.index,
    title="Distribución de Productos por Tipo de Cliente",
    labels={"x": "Proporción", "y": "Producto", "color": "Tipo de Cliente"},
    color_discrete_sequence=px.colors.qualitative.Set3
)

# Configurar el gráfico como barras apiladas
fig_1.update_layout(barmode='stack', xaxis_title="Proporción", yaxis_title="Producto")

fig_1.show()

#### **Dependencia entre Producto y Satisfacción**

In [48]:
# Tabla de contingencia entre 'Producto' y 'Satisfacción'
tabla_contingencia_satisfaccion = pd.crosstab(df['Producto'], df['Satisfaccion'])

# Realizar la prueba de Chi-cuadrado para independencia
chi2, p, dof, expected = chi2_contingency(tabla_contingencia_satisfaccion)

# Resultados de la prueba
print(f"Chi-cuadrado: {chi2:.4f}")
print(f"Valor p: {p:.4f}")
print(f"Grados de libertad (dof): {dof}")
print("Valores esperados:\n", expected)

if p < 0.05:
    print("Se rechaza la hipótesis nula: Hay dependencia entre las variables.")
else:
    print("No se rechaza la hipótesis nula: No hay evidencia de dependencia entre las variables.")

Chi-cuadrado: 5.1795
Valor p: 0.2694
Grados de libertad (dof): 4
Valores esperados:
 [[18.36  3.6  14.04]
 [16.83  3.3  12.87]
 [15.81  3.1  12.09]]
No se rechaza la hipótesis nula: No hay evidencia de dependencia entre las variables.


In [49]:
# Crear la tabla de contingencia normalizada por índice
tabla_contingencia_satisfaccion = pd.crosstab(df['Producto'], df['Satisfaccion'], normalize='index')

# Crear el gráfico de barras
fig_2 = px.bar(
    tabla_contingencia_satisfaccion,
    x=tabla_contingencia_satisfaccion.columns,
    y=tabla_contingencia_satisfaccion.index,
    title="Distribución de Productos por Satisfacción del Cliente",
    labels={"x": "Proporción", "y": "Producto", "color": "Satisfacción del Cliente"},
    color_discrete_sequence=px.colors.qualitative.Set3
)

# Configurar el gráfico como barras apiladas
fig_2.update_layout(barmode='stack', xaxis_title="Proporción", yaxis_title="Producto")

fig_2.show()

#### **Distribución de Poisson**

In [50]:
# Parámetro lambda (media de Unidades Vendidas)
lambda_poisson = df['Unidades_Vendidas'].mean()

# Generar valores de la distribución de Poisson
x = np.arange(0, df['Unidades_Vendidas'].max() + 1)
poisson_probs = (lambda_poisson**x * np.exp(-lambda_poisson)) / factorial(x)

# Gráfico de la distribución de Poisson
fig = go.Figure(data=[go.Bar(x=x, y=poisson_probs, marker_color=px.colors.qualitative.Set3[0])])

fig.update_layout(
    title_text='Distribución de Poisson de Unidades Vendidas',
    xaxis_title='Unidades Vendidas',
    yaxis_title='Probabilidad'
)

fig.show()

# Mostrar la media y varianza
print(f"Media de unidades Vendidas (Lambda): {lambda_poisson:.4f}")
print(f"Varianza de Unidades Vendidas: {lambda_poisson:.4f}")

Media de unidades Vendidas (Lambda): 5.0100
Varianza de Unidades Vendidas: 5.0100


### **Conclusiones**

**Optimización de inventarios:** Los resultados obtenidos en este análisis preliminar sugieren que un modelo de demanda basado en la distribución de Poisson puede ser una herramienta valiosa para optimizar los niveles de inventario en la tienda. Al aplicar este modelo a datos reales, podremos realizar pronósticos más precisos de las ventas, lo que permitirá ajustar los niveles de inventario de forma dinámica y reducir los costos asociados a la sobreproducción o a la escasez de productos.

**Perfil del cliente:** El análisis exploratorio ha revelado patrones interesantes en el comportamiento de compra de los clientes. Al segmentar a los clientes en función de su tipo y preferencias, podremos diseñar estrategias de marketing más personalizadas y efectivas. Por ejemplo, podemos ofrecer promociones especiales a los clientes recurrentes para incentivar la compra de productos de mayor valor.

**Gestión eficiente:** La implementación de un sistema de gestión basado en datos permitirá a la tienda tomar decisiones más informadas y basadas en evidencia. Al monitorear continuamente las ventas y ajustar los modelos predictivos, podremos identificar nuevas oportunidades de crecimiento y optimizar la asignación de recursos.